In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pricing import EquityForwardCurve, ForwardVariance, DiscountingCurve,piecewise_function
from scipy import exp, sqrt, log, heaviside
from targetvol import optimal_strategy

### Two equities

#### market data

In [2]:
N_equity = 2                                #number of equities
T_max = 10
t = 0.                                      #pricing date
spot_price = np.array([110,120])            #spot price of the two assets
corr = np.array(([1.,0.15],[0.15,1.]))        #correlation matrix

"""Market Discount factors"""
r_t = np.array([1.,0.9,1.2])/100            
T_discounts = np.array([0.1,1.,T_max])      #data observation of the market discounts factor
market_discounts = exp(-r_t*T_discounts)       #market discounts factor

"""Market repo rates"""  #https://tradingeconomics.com/united-states/repo-rate
T_repo1 = np.array([1.2,3.,5.3,T_max])       #data observation of the market repo rates for equity 1
repo_rate1 = np.array([5.875,5.50,2.,2.3])/100  #market repo rates for equity 1

T_repo2 = np.array([2.,3.,4.,T_max])
repo_rate2 = np.array([2.,3.,3.034,2.99])/100

for i in range(N_equity):                   #dimensional check between time and market data
    a = globals()["repo_rate" + str(i+1)]
    b = globals()["T_repo" + str(i+1)]
    if(a.shape!=b.shape):
        raise Exception("WARNING: Different dimension between data and value")
        
"""Spot Volatilities"""
sigma1 = np.array([32.2,29.3,29.1,29.5,29.7,29.8,29.9,30.6,30.62])/100
T_sigma1 = np.array([0.08,0.17,0.25,0.33,0.42,0.5,1.,2.,T_max])
K1 = np.array([spot_price[0],200])
spot_vola1 = np.array((sigma1,sigma1))                                      #market implied volatility matrix

sigma2 = np.array([35.,31.,31.,31.4,31.3,30.7,29.2,28.9,28.88])/100
T_sigma2 = np.array([0.08,0.17,0.25,0.33,0.42,0.5,1.,2.,T_max])
K2 = np.array([spot_price[1],200])
spot_vola2 = np.array((sigma2,sigma2))      

for i in range(N_equity):                   #dimensional check between time and market data
    a = globals()["sigma" + str(i+1)]
    b = globals()["T_sigma" + str(i+1)]
    if(a.shape!=b.shape):
        raise Exception("WARNING: Different dimension between data and value")

#### Loading Discouting curve, Forward curve and forward variance matrix

In [3]:
D = DiscountingCurve(reference=t, discounts=market_discounts,dates=T_discounts)
F = np.array([])
V = np.array([])
for i in range(N_equity):
    q = globals()["repo_rate" + str(i+1)]
    T_q = globals()["T_repo" + str(i+1)]
    s_vola = globals()["spot_vola" + str(i+1)]
    T_vola = globals()["T_sigma" + str(i+1)]
    K = globals()["K" + str(i+1)]
    F = np.append(F,EquityForwardCurve(reference=t,spot=spot_price[i],discounting_curve=D,repo_dates=T_q,repo_rates=q))
    V = np.append(V,ForwardVariance(reference=t,spot_volatility=s_vola,strikes=K,maturities=T_vola,strike_interp=spot_price[i]))

Forward zero rate:  [0.01       0.00888889 0.01233333]
Forward volatility:  [0.322      0.26456694 0.28670368 0.30716445 0.30422086 0.30319589
 0.29999667 0.31284341 0.30624998]
Forward volatility:  [0.35       0.26950572 0.31       0.326184   0.30930567 0.27334776
 0.27618653 0.28596853 0.28874998]


In [4]:
tau, alpha = optimal_strategy(F,V,corr,1e6)

Strategy time grid:  [ 0.08  0.17  0.25  0.33  0.42  0.5   1.    1.2   2.    3.    4.    5.3
 10.  ]
[0.77203721 0.79808043 0.57516338 ... 0.81054361 0.77903941 0.42671362]
[0.30724052 0.45662158 0.55538837 ... 0.37060564 0.54610212 0.55103752]
[0.57755723 0.48812415 0.43032008 ... 0.39396298 0.25349612 0.61491864]
[0.54305937 0.52024358 0.27833275 ... 0.29689867 0.44452286 0.62360027]
[0.38019963 0.63490994 0.62778936 ... 0.59793868 0.58170008 0.58288025]
[0.62320097 0.50899122 0.64978998 ... 0.63678546 0.64991565 0.64265454]
[0.50802694 0.55513938 0.54430418 ... 0.54907776 0.58102614 0.55548746]
[0.3435592  0.54176341 0.44680187 ... 0.53403882 0.25228291 0.50497219]
[0.61195525 0.61166538 0.59152945 ... 0.59249396 0.50356288 0.60794173]
[0.31333531 0.37588651 0.33142461 ... 0.36123591 0.37376844 0.3587764 ]
[0.37119513 0.3349937  0.2385961  ... 0.36843    0.32931242 0.32702252]
[0.38180204 0.36319015 0.29410354 ... 0.35017841 0.32037082 0.33931493]
[nan nan nan ... nan nan nan]


/home/ste/Desktop/Strategy/targetvol.py:33: RuntimeWarning: divide by zero encountered in true_divide
  f = np.dot(alpha,mu[i,:])/np.linalg.norm(np.dot(alpha,nu[:,:,i]),axis=1)  #score function vector
/home/ste/Desktop/Strategy/targetvol.py:33: RuntimeWarning: invalid value encountered in true_divide
  f = np.dot(alpha,mu[i,:])/np.linalg.norm(np.dot(alpha,nu[:,:,i]),axis=1)  #score function vector
